### In this notebook we go through the process of training our model

We'll start by loading our data and encoding our categorical variables,  
since in our EDA process we saw that our data contains many unique values,  
it's best to use label encoding over one-hot encoding

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#from sklearn.preprocessing import LabelEncoder

#artist_enc = LabelEncoder()
#genre_enc = LabelEncoder()

ag_df = pd.read_csv('data/ag_data.csv')
ag_df.head()

,Artist,Subgenre
0,Jason Mraz,acoustic pop
1,Jason Mraz,dance pop
2,Jason Mraz,neo mellow
3,Jason Mraz,pop
4,The Paper Kites,acoustic pop


In [2]:
ag_df2 = ag_df[~(ag_df['Subgenre'].str.contains('sleep'))].copy()
#ag_df2['Artist'] = artist_enc.fit_transform(ag_df2['Artist'])
#ag_df2['Subgenre'] = genre_enc.fit_transform(ag_df2['Subgenre'])

ag_df2.head()

,Artist,Subgenre
0,Jason Mraz,acoustic pop
1,Jason Mraz,dance pop
2,Jason Mraz,neo mellow
3,Jason Mraz,pop
4,The Paper Kites,acoustic pop


In [3]:
ag_df.shape

(83983, 2)

Now that our data is preprocessed we can begin with training our model

In [4]:
from kmodes.kmodes import KModes

km_model = KModes(n_clusters=300)

In [5]:
km_model.fit(ag_df2)